In [19]:
import numpy as np
import pandas as pandas
import math
import cv2
from torch.utils.data import Dataset, DataLoader
from generate_inputs import RelativePose
import torch

In [85]:
class TrajectoryDataset(Dataset):

    def __init__(self, root_dir, traj_num_list , max_num_pose, max_dis, max_ang):
        idx_map = np.zeros(len(traj_num_list), dtype = int)
        rel_pose_idx_map = []
        rel_pose_map = []
        l = 0
        for k in traj_num_list:
            dir = root_dir + 'traj_' + str(k) + '/'
            print(k)

            pose_list = pandas.read_csv(dir + 'pose_list.csv', header=None).values
            N = len(pose_list)
            rel_pose_idx_list = np.zeros(N-1, dtype = int)
            rel_pose_list = []

            for i in range(0, N-1):
                num_poses = 0
                for j in range(i+1, N, 1):
                    rel_pose = RelativePose(pose_list[i], pose_list[j])
                    dis = np.sqrt(rel_pose[0]*rel_pose[0] + rel_pose[1]*rel_pose[1])
                    ang = rel_pose[2]

                    if dis <= max_dis and ang <= max_ang and (j - i) <= max_num_pose:
                        rel_pose_list.append(rel_pose)
                        num_poses = num_poses + 1
                    else:
                        break      
                rel_pose_idx_list[i] = num_poses
            rel_pose_idx_list = np.cumsum(rel_pose_idx_list)
            idx_map[l] = len(rel_pose_list)

            if rel_pose_idx_list[-1] != idx_map[l]:
                print('Error in TrajectoryDataset')
            rel_pose_idx_map.append(rel_pose_idx_list)
            rel_pose_map.append(rel_pose_list)
            l = l + 1
        idx_map = np.cumsum(idx_map)
        
        self.idx_map = idx_map
        self.rel_pose_idx_map = rel_pose_idx_map
        self.rel_pose_map = rel_pose_map
        self.root_dir = root_dir
        self.traj_num_list = traj_num_list

    def __len__(self):
        return self.idx_map[-1]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        if idx >= self.idx_map[-1] or idx < 0:
            print('Larger than bound')

        outer_idx = np.argmax(self.idx_map > idx)
        if outer_idx > 0:
            ij = idx - self.idx_map[outer_idx - 1]
            inner_idx = np.argmax(self.rel_pose_idx_map[outer_idx] > ij)
        else:
            ij = idx
            inner_idx = np.argmax(self.rel_pose_idx_map[outer_idx] > ij)

        i = inner_idx
        if i > 0:
            j = ij - self.rel_pose_idx_map[outer_idx][i - 1] + i + 1
        else:
            j = ij + i + 1
        
        dir = self.root_dir + 'traj_' + str(self.traj_num_list[outer_idx]) + '/'
        
        image1 = cv2.imread(dir + 'img_' + str(i) + '.jpg', 0)/255
        image2 = cv2.imread(dir + 'img_' + str(j) + '.jpg', 0)/255
        rel_pose = self.rel_pose_map[outer_idx][ij]
        
        sample = {'image1': image1, 'image2': image2, 'rel_pose': rel_pose}

        return sample


In [86]:
train_dataset = TrajectoryDataset('./data/', np.arange(1,11) , 50, 0.5, math.radians(10))

1
2
3
4
5
6
7
8
9
10


In [88]:
len(train_dataset)

112576

In [90]:
train_dataset[112575]['image2']

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
58301 - 44201

14100

In [ ]:
import numpy as np
import pandas as pd
from generate_inputs import RelativePose
import math

traj_num_list = np.array([1,3,8])
root_dir = './data/'
max_dis = 0.5
max_ang = math.radians(10)
max_num_pose = 50

idx_map = np.zeros(len(traj_num_list), dtype = int)
rel_pose_idx_map = []
rel_pose_map = []
l = 0
for k in traj_num_list:
    dir = root_dir + 'traj_' + str(k) + '/'
    print(k)
    
    pose_list = pd.read_csv(dir + 'pose_list.csv', header=None).values
    N = len(pose_list)
    rel_pose_idx_list = np.zeros(N-1, dtype = int)
    rel_pose_list = []
    
    for i in range(0, N-1):
        num_poses = 0
        for j in range(i+1, N, 1):
            rel_pose = RelativePose(pose_list[i], pose_list[j])
            dis = np.sqrt(rel_pose[0]*rel_pose[0] + rel_pose[1]*rel_pose[1])
            ang = rel_pose[2]

            if dis <= max_dis and ang <= max_ang and (j - i) <= max_num_pose:
                rel_pose_list.append(rel_pose)
                num_poses = num_poses + 1
            else:
                break      
        rel_pose_idx_list[i] = num_poses
    rel_pose_idx_list = np.cumsum(rel_pose_idx_list)
    idx_map[l] = len(rel_pose_list)
    
    if rel_pose_idx_list[-1] != idx_map[l]:
        print('Error in TrajectoryDataset')
    rel_pose_idx_map.append(rel_pose_idx_list)
    rel_pose_map.append(rel_pose_list)
    l = l + 1
idx_map = np.cumsum(idx_map)